In [ ]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from dokdonia import differentialexpression as DE
from dokdonia import visualization as VI
from dokdonia import clusteranalysis as CA
from dokdonia.pathway import KEGGPathwayParser, GenomeGBK, add_pathways_to_deseq_df, show_pathways_in_ranked_genes
from dokdonia.utils import take_average_values, saveToPickleFile, readFromPickleFile

%matplotlib inline


root_dir = Path(os.getcwd())
results_dir = root_dir / "results"
data_dir = root_dir / "data"

## Load counts

In [ ]:
min_count = 10

counts = pd.read_csv(data_dir / 'counts' / 'DokdoniaCounts.csv', index_col=0)
counts = counts[counts.filter(regex='^[^T]+$').columns]
conditions = [name.split('.sam')[0] for name in counts.columns]
counts.columns = conditions
counts = counts[(counts > min_count).all(1)]
counts.reset_index(level=0, inplace=True)

## Load KEGG pathways and genome annotations

In [ ]:
gbk = GenomeGBK(data_dir / 'genome' / 'DokdoniaMED134.gbk')

KEGGparser = KEGGPathwayParser.fromKEGGidentifier('dok', only_curated_pathways=True)
gene_pathways, gene_systems = KEGGparser.getGenePathways()
system_pathways = KEGGparser.getSystemPathways()
# gene_info = KEGGparser.getGeneInfoFromKEGGorthology()
gene_list = list(gene_pathways.keys())
print(f'There are a total of {len(gene_list)} genes')

## Get DeSeq2 normalized counts

In [ ]:
# Deseq2 normalization
colfactor = pd.DataFrame(
    {'Sample': counts.columns, 'Temperature': counts.columns.str.extract(r'_(\d+)_', expand=False)}
    ).iloc[1:, :].set_index('Sample', inplace=False)
colfactor.head()

deseq2_counts = DE.deseq2Normalize(counts, colfactor,
                gene_column="index",
                design_formula="~ Temperature")
deseq2_counts.to_csv(data_dir / "processed" / "DokdoniaMED134_DS2.tsv", sep="\t", index=False)
deseq2_counts.head()

## Compute Transcript / cell values

In [ ]:
sample_meta = pd.read_excel(data_dir / "normalization" / "Datos_Dokdonia_9Jun23.xlsx")
sample_meta["Sample"] = sample_meta['Light/Dark'] + '_' + sample_meta['Temperature'].astype(str) + '_' + sample_meta['Replicate']
sample_meta.head()

In [ ]:
TC = DE.get_transcript_cell(counts, sample_meta, ["D_25_R1"]).set_index("index")
TC.to_csv(data_dir / "processed" / "DokdoniaMED134_TC.tsv", sep="\t")
TC.head()

## Compare median values between counts, DeSeq2-normalized counts, and transcript / cell values

In [ ]:
# Take median values across replicates
counts_avg = take_average_values(counts.set_index("index")).median()
TC_avg = take_average_values(TC, method='median')
DS2_avg = take_average_values(deseq2_counts.set_index("index")).median()

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 6))

ax0 = counts_avg.plot(kind='bar', ax=axes[0], title='Raw counts')
ax0.set_xlabel('Temperature')
ax0.set_ylabel('Median raw counts')

ax1 = DS2_avg.plot(kind='bar', ax=axes[1], title='DeSeq2 normalized counts')
ax1.set_xlabel('Temperature')
ax1.set_ylabel('Median DeSeq2-norm counts')

ax2 = TC_avg.median().plot(kind='bar', ax=axes[2], title='Transcripts/cell')
ax2.set_xlabel('Temperature')
ax2.set_ylabel('Median transcripts/cell')

plt.tight_layout()
plt.show()

## Remove Light/Dark DE genes from datasets

In [ ]:
DE_all_T = readFromPickleFile(results_dir / "deseq_results" / "DE_all_T.pkl")
DE_genes_across_T = readFromPickleFile(results_dir / "deseq_results" / "DE_genes_across_T.pkl")

# Remove light-dark DE genes from counts
counts_noDE = counts.loc[(
    (~counts["index"].isin(DE_all_T))
    )]

# Remove light-dark DE genes from Deseq2 dataset
deseq2_counts_noDE = deseq2_counts.loc[(
    (~deseq2_counts["index"].isin(DE_all_T))
    )]

# Remove light-dark DE genes from TC dataset
TCnoDE = TC.loc[(
    (~TC.index.isin(DE_all_T))
    )]

# Find clusters based on expression pattern across temperatures: DeSeq2-normalized counts

In [ ]:
# Using Deseq2 data
clust_tightness = 3
res_id = 'CLUSTER_ONLY_TEMP_DE_GENES_DESEQ2_ZSCORES'
workdir = os.path.join(os.getcwd(), data_dir / 'clust_input')
outdir = os.path.join(os.getcwd(), results_dir / 'clust' / res_id)

clusters_DS2Z = CA.getGeneClusters(deseq2_counts_noDE.set_index("index"),path_to_wd=workdir, 
                              out_dir=outdir,
                              cluster_tightness=clust_tightness,
                              normalization_file='clust_normalization_only_zscores.txt',
                              replicates_file='clust_replicates_merged_L_D_volume.txt',
                              scaling_factor=1)

# Plot clusters
plot_cluster_data_DS2Z = pd.read_csv(os.path.join(
    os.getcwd(),results_dir / "clust" / f'{res_id}/Processed_Data/clust_input.tsv_processed.tsv'),
    sep='\t', index_col='Genes')

In [ ]:
# Print total number of genes contained in the sum of all clusters
total_genes_in_clusters = [g for cluster in clusters_DS2Z.values() for g in cluster]
print(f'Total number of genes in all clusters: {len(total_genes_in_clusters)}')
# Print fraction of total genes in cluisters versus total number of genes
print(f'Fraction of total genes in clusters: {len(total_genes_in_clusters) / len(plot_cluster_data_DS2Z)}')

In [ ]:
# Rename clusters to group by pattern
cluster_map = {
    "C0": "C0",
    "C1": "C2",
    "C2": "C3",
    "C3": "C1",
    "C4": "C4"
}

clusters_DS2Z = {
    cluster_map[cluster_id]: cluster
    for cluster_id, cluster in clusters_DS2Z.items()
    if cluster_id != "C5"
}

clusters_DS2Z = dict(sorted(clusters_DS2Z.items(), key=lambda x: x[0]))

# Save figure data
saveToPickleFile(clusters_DS2Z, results_dir / "figures" / "figure_data" / "clusters_DS2Z.pkl")
saveToPickleFile(plot_cluster_data_DS2Z, results_dir / "figures" / "figure_data" / "plot_cluster_data_DS2Z.pkl")


In [ ]:
VI.plotClusters(plot_cluster_data_DS2Z, clusters_DS2Z)

## Find genes in clusters that are not DE according to the LRT test of Deseq2

There are about 790 DE genes across temperatures according to LRT, alpha=0.01, but over 1200 within the clusters. What happens with the genes that are not DE according to LRT but are in the clusters and do follow non random patterns?

In [ ]:
print(f"There are {len(DE_genes_across_T)} genes that are differentially expressed across all temperatures.")

In [ ]:
# DE_genes_across_T
clusters_DS2Z_noDE = {
    cluster_id: [g for g in cluster if g not in DE_genes_across_T]
    for cluster_id, cluster in clusters_DS2Z.items()
}

# Genes that are not DE across temperatures AND not included in any cluster
not_in_clusters_and_no_DE = [
    g for g in deseq2_counts_noDE.set_index("index").index
    if g not in DE_genes_across_T and g not in total_genes_in_clusters
]

print(f"There are {len(not_in_clusters_and_no_DE)} genes that are not DE across temperatures AND not included in any cluster.")

## Plot genes that are included in clusters but are not DE across temperatures

In [ ]:
VI.plotClusters(plot_cluster_data_DS2Z, clusters_DS2Z_noDE)

## Annotate and rank genes within clusters

In [ ]:
res_id = "CLUSTER_ONLY_TEMP_DE_GENES_DESEQ2_ZSCORES"

cluster_data = pd.read_csv(os.path.join(
    os.getcwd(),results_dir / "clust" / f'{res_id}/Input_files_and_params/Data/clust_input.tsv'),
    sep='\t', index_col='index')
ranked_clusters_avg_expr = CA.rankGenesWithinClusters(clusters_DS2Z, cluster_data, method="median")

In [ ]:
if not (results_dir / "pathways").exists():
   (results_dir / "pathways").mkdir(exist_ok=False)

no_kegg_pathway = []
ranked_clusters = []
for cluster_id in ranked_clusters_avg_expr:
    ranked_df = show_pathways_in_ranked_genes(
        ranked_clusters_avg_expr[cluster_id],
        gbk, gene_pathways,
        gene_systems, n=None
        )
    no_kegg_pathway.append( 100 * (ranked_df[((ranked_df.subsystem.str.contains("Unspecified")) & ~ ranked_df.subsystem.isna())].shape[0]) / ranked_df.shape[0] )
    ranked_df.insert(0, "cluster", cluster_id)
    ranked_df.to_csv(results_dir / "pathways" / f"ranked_{cluster_id}.csv")
    ranked_clusters.append(ranked_df)

merged_ranked_clusters = pd.concat(ranked_clusters).sort_values(by="value", ascending=False)
merged_ranked_clusters.to_csv(results_dir / "pathways" / "ranked_clusters.csv")
print(no_kegg_pathway)
print(np.mean(no_kegg_pathway))

## Ranked lists among the global 10% by expression

In [ ]:
percent_cutoff = 10

filtered_ranked_clusters = {}

all_genes_expression = {gene: expr for cluster in ranked_clusters_avg_expr.values() for gene, expr in cluster.items()}

sorted_all_genes_expression = sorted(all_genes_expression.items(), key=lambda x:x[1], reverse=True)
sorted_all_genes_expression = dict(sorted_all_genes_expression)

cutoff_position = int( (10 / 100) * len(sorted_all_genes_expression) )
cutoff_value = list(sorted_all_genes_expression.values())[cutoff_position]
print(f"Cutoff average expression value of: {cutoff_value} Deseq2 values")
top_genes = list(sorted_all_genes_expression.keys())[:cutoff_position]

for cluster_id, cluster in ranked_clusters_avg_expr.items():
    filtered_ranked_clusters[cluster_id] = {gene: expr for gene, expr in cluster.items() if gene in top_genes}

In [ ]:
# Cluster IDs in the top 10% of genes
top_10_ranked = merged_ranked_clusters.loc[merged_ranked_clusters.value > cutoff_value, :]
top_10_ranked.cluster.value_counts()

Genes in the top 10% by expression (DeSeq2-normalized) tend to located in clusters with a downward pattern, negativaly correlated with temperature (clusters C1 and C4.)

## Rank temperature - independent genes and assign pathways

In [ ]:
from dokdonia.utils import sort_dict_by_values

deseq2_avg = take_average_values(deseq2_counts.set_index("index"), method="median")

temp_indep_ranked_genes = {
    gene_id: deseq2_avg.loc[gene_id].median()
    for gene_id in not_in_clusters_and_no_DE
}

temp_indep_ranked_genes = sort_dict_by_values(temp_indep_ranked_genes, reverse=True)

# Add pathways
temp_indep_pathways = show_pathways_in_ranked_genes(
    temp_indep_ranked_genes, gbk, gene_pathways, gene_systems, n=None
    )

# Save to csv in pathways folder
temp_indep_pathways.to_csv(
    results_dir / "pathways" / "temp_indep_pathways.csv"
    )

temp_indep_pathways.head()

In [ ]:
temp_indep_pathways.loc[temp_indep_pathways.value > cutoff_value, "system"].value_counts()

## Find fraction of unannotated genes in clusters

As a side note, which fraciton of genes in Dokdonia MED134 are unannotated??

In [ ]:
lines = ["gene_id\tgene_name\tproduct\tkegg_pathway\n"]
for gene_id in counts["index"]:   
    gene_info = gbk.getGeneInfo(gene_id)
    gene_name = gene_info["gene"][0] if "gene" in gene_info else "unspecified"
    product = gene_info["product"][0]
    kegg_pathway = ",".join(gene_pathways[gene_id]) if gene_id in gene_pathways else "unspecified"
    lines.append(
        (
            f"{gene_id}\t"
            f"{gene_name}\t"
            f"{product}\t"
            f"{kegg_pathway}\n"
            )
        )
with open(results_dir / "pathways" / "gene_annotations.tsv", "w") as file:
    file.writelines(lines)


annot = pd.read_csv(results_dir / "pathways" / "gene_annotations.tsv", sep="\t").set_index("gene_id")

print(f"Fraction of genes with no KEGG annotation: {sum(annot.kegg_pathway == 'unspecified') / len(annot):.2%}")
print(f"Fraction of genes with no product annotation: {sum(annot['product'] == 'hypothetical protein') / len(annot):.2%}")
print(f"Fraction of genes with no gene name annotation: {sum(annot['gene_name'] == 'unspecified') / len(annot):.2%}")

kegg_cluster_repr = []
for cluster_id, cluster_genes in clusters_DS2Z.items():
    kegg_cluster_repr.append(
        sum(annot.loc[cluster_genes].kegg_pathway == "unspecified") / len(cluster_genes)
    )
# Print average fraction of genes with no KEGG annotation per cluster
print(f"Average fraction of genes with no KEGG annotation per cluster: {np.mean(kegg_cluster_repr):.2%}")


## Enriched metabolic pathways in Deseq2 clusters

In [ ]:
from dokdonia.utils import terminal_execute

print(res_id)
out_dir = results_dir / "enrichment_results"
out_dir.mkdir(exist_ok=True)

clusters_path = results_dir / "clust" / f"{res_id}" / "Clusters_Objects.tsv"
stout = terminal_execute(
    command_str=(
    f"Rscript {root_dir / 'dokdonia' / 'clusterProfiler.R'}"
    f" {res_id} {clusters_path} {out_dir}"
    )
)

# Show enriched pathways
enrichment_results = pd.read_csv(out_dir / f"results_{res_id}.csv")

enrichment_results.Cluster = enrichment_results.Cluster.apply(lambda x: cluster_map[f"C{int(list(x)[-1]) - 1}"])
enrichment_results.set_index("Cluster", inplace=True)

# Write to csv file
enrichment_results.to_csv(out_dir / f"results_{res_id}.csv")
enrichment_results.head()